In [19]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [20]:
# Get data from the link
csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'

data = pd.read_csv(csv_url, sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Simple inspection

In [21]:
# with pandas
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [22]:
# Generate the profile report with Pandas Profiling
profile = ProfileReport(
    data,
    title="Example of summarization of wine data"
)


In [23]:
# Generate the dataset profile
# This is a nice and simple way to document the data
#profile.to_notebook_iframe()

## Unit tests

### Basic examples - function tests

We will learn how the unit test work on  a simple function. First, we will define a function `square`, which returns the square of a number. Then, we will test it by writing assertions (correct answers) in a test function. 

In [24]:
import pytest

# install the following to be able to run the tests in notebook
import ipytest
ipytest.autoconfig()

In [25]:
# A simple function: calculate square of a number
def square(x):
    return x * x

In [26]:
%%run_pytest[clean]

# Let's test the function
# Think about the limit cases
def test_square():
    assert square(2) == 4
    assert square(0) == 0
    assert square(-2) == 4


.                                                                                            [100%]
1 passed in 0.00s


%%run_pytest[clean] and %%run_pytest are deprecated in favor of %%ipytest. %%ipytest will clean tests, evaluate the cell and then run pytest. To disable cleaning, configure ipytest with ipytest.config(clean=False).


Make the test fail to be sure to understand how it works.

### Basic examples - data tests

As we did for the function, we can also write assertions for the data. In the following example we will define a data frame on the fly and thest for the null values in it. 

In [27]:
%%run_pytest[clean]

def test_column_is_null():
    df = pd.DataFrame(data = [(1, 0), (2, None)],
                      columns = ['a', 'b'])
    
    assert np.all(pd.notna(df))

F                                                                                            [100%]
============================================= FAILURES =============================================
_______________________________________ test_column_is_null ________________________________________

    def test_column_is_null():
        df = pd.DataFrame(data = [(1, 0), (2, None)],
                          columns = ['a', 'b'])
    
>       assert np.all(pd.notna(df))
E       assert False
E        +  where False = <function all at 0x7f6890029ef0>(      a      b\n0  True   True\n1  True  False)
E        +    where <function all at 0x7f6890029ef0> = np.all
E        +    and         a      b\n0  True   True\n1  True  False = <function notna at 0x7f6858e0b320>(   a    b\n0  1  0.0\n1  2  NaN)
E        +      where <function notna at 0x7f6858e0b320> = pd.notna

/tmp/ipykernel_159316/1936835947.py:5: AssertionError
===================================== short test summary info ===========

%%run_pytest[clean] and %%run_pytest are deprecated in favor of %%ipytest. %%ipytest will clean tests, evaluate the cell and then run pytest. To disable cleaning, configure ipytest with ipytest.config(clean=False).


## Test the wine data

Previously, we generated the data frame inside the test function. If we want to run multiple tests on the same df, we would rather pass it to each function as an argument (as usual in programming). To do that in testing, we need to define the data as **fixtures**. They look like ordinary function definitions, preceeded by a decorator `@pytest.fixture`. 

### Raw data tests

In [28]:
# Define fixtures
@pytest.fixture
def input_schema():
    # Define range and type for each column
    schema = {
    'fixed acidity': {'min': 1.0, 'max': 17.0, 'type': float},
    'volatile acidity': {'min': 0.0, 'max': 2.0, 'type': float},
    'citric acid': {'min': 0.0, 'max': 2.0, 'type': float},
    'residual sugar': {'min': 0.5, 'max': 17.0, 'type': float},
    'chlorides': {'min': 0.0, 'max': 17.0, 'type': float},
    'free sulfur dioxide': {'min': 0.0, 'max': 80.0, 'type': float},
    'total sulfur dioxide': {'min': 0.0, 'max': 300.0, 'type': float},
    'density': {'min': 0.8, 'max': 1.1, 'type': float},
    'pH': {'min': 1.0, 'max': 10.0, 'type': float},
    'sulphates': {'min': 0.0, 'max': 2.0, 'type': float},
    'alcohol': {'min': 7.0, 'max': 17.0, 'type': float},
    'quality': {'min': 1, 'max': 10, 'type': int},
    }
    return schema


# Download the data
@pytest.fixture
def input_data():
    csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    data = pd.read_csv(csv_url, sep=';')
    return data

Write the following tests:
- is the number of columns in the data frame the same as in schema definition?
- are the values within defined ranges?
- are the types of the columns correct?

In [29]:
%%run_pytest[clean]

def test_number_of_columns(input_data, input_schema):
    
    assert len(input_data.columns) == len(input_schema.keys())


def test_input_data_ranges(input_data, input_schema):
    
    for column in input_data.columns:
        min_val = input_data.loc[:, column].min()
        max_val = input_data.loc[:, column].max()

        min_schema = input_schema[column]['min']
        max_schema = input_schema[column]['max']
        
        #assert True
        #assert False
        assert min_val >= min_schema
        assert max_val <= max_schema
        
        
def test_input_types(input_data, input_schema):
    
    for column in input_data.columns:
        input_type = input_data.dtypes[column]
        schema_type = input_schema[column]['type']
        
        assert input_type == schema_type
    

%%run_pytest[clean] and %%run_pytest are deprecated in favor of %%ipytest. %%ipytest will clean tests, evaluate the cell and then run pytest. To disable cleaning, configure ipytest with ipytest.config(clean=False).


...                                                                                          [100%]
3 passed in 4.43s


### Feature engineering tests

**NOTE:** Data transformaton should be done only on test dataset. You fit the transformer on the test dataset and then apply it on the train dataset. Since we are only illustrating the functioning of the unit testing, we will do it on the whole dataset.

In [30]:
from sklearn.preprocessing import StandardScaler
from numpy import mean, std

In [31]:
# Let's transform a column...

# define standard scaler
scaler = StandardScaler()
# transform data
scaled = scaler.fit_transform(data[['alcohol']])
print(scaled)

[[-0.96024611]
 [-0.58477711]
 [-0.58477711]
 ...
 [ 0.54162988]
 [-0.20930812]
 [ 0.54162988]]


In [32]:
# And check the stats...
print('mean:', mean(scaled))
print('std:', std(scaled))

mean: 1.0664806540489309e-16
std: 1.0


In [33]:
@pytest.fixture
def scaled_alcohol():
    csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    data = pd.read_csv(csv_url, sep=';')
    
    # Define scaler
    scaler = StandardScaler()
    # Transform data
    scaled = scaler.fit_transform(data[['alcohol']])
    return scaled

In [34]:
%%run_pytest[clean]
# Test: is mean around zero and std around one?

def test_scaled_mean_zero(scaled_alcohol):
    
    mean_val = mean(scaled_alcohol)
    std_val = std(scaled_alcohol)
    
    assert pytest.approx(mean_val) == 0.0
    assert pytest.approx(std_val) == 1.0


%%run_pytest[clean] and %%run_pytest are deprecated in favor of %%ipytest. %%ipytest will clean tests, evaluate the cell and then run pytest. To disable cleaning, configure ipytest with ipytest.config(clean=False).


.                                                                                            [100%]
1 passed in 1.17s


## Additional exercises:

- implement and Test MinMaxScaler
- test null on 'quality'
- repeat the tests on synthetic data

In [35]:
%%run_pytest[clean]

def test_number_of_columns(input_data, input_schema):
    
    print(len(input_data.columns))
    #assert len(input_data.columns) == len(input_schema)

%%run_pytest[clean] and %%run_pytest are deprecated in favor of %%ipytest. %%ipytest will clean tests, evaluate the cell and then run pytest. To disable cleaning, configure ipytest with ipytest.config(clean=False).


.                                                                                            [100%]
1 passed in 1.20s
